In [318]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from python_scripts.utils.feature_selection import *
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

# Feature Extraction

In [319]:
final_dataset_path = '../../dataset/final/windows/'
acc_dataset_path = final_dataset_path + 'PatchTable_Acceleration_Normalised.csv'
gyro_dataset_path = final_dataset_path + 'PatchTable_Gyroscope_Normalised.csv'

In [320]:
acc_dataset_df = pd.read_csv(acc_dataset_path)
gyro_dataset_df = pd.read_csv(gyro_dataset_path)
acc_dataset_df = acc_dataset_df.drop('ClassificationType', axis=1)
gyro_dataset_df = gyro_dataset_df.drop('ClassificationType', axis=1)

In [321]:
acc_dataset_df

,Subject,Class,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
0,1,0,0.617654,0.747748,0.597720,0.508189,-0.277193,-0.163081,-0.438910,-0.414596,...,-0.651852,-0.165272,-0.651852,-0.285245,-0.146060,-0.086591,-0.206281,-1.771846,0.959438,-0.652570
1,1,0,-1.392837,1.610635,1.380251,0.597923,-0.279359,-0.163483,-0.442177,-0.421079,...,-0.659200,-0.224007,-0.659200,-0.285264,-0.146061,0.952947,0.737416,-2.360037,0.966257,-0.659603
2,1,0,-1.567663,1.487365,1.268460,0.508189,-0.279558,-0.163548,-0.442086,-0.421402,...,-0.659522,-0.224137,-0.659522,-0.285264,-0.146061,1.178809,0.885754,-2.654132,0.966257,-0.659652
3,1,0,1.579194,1.610635,1.268460,0.508189,-0.279463,-0.163525,-0.441743,-0.421428,...,-0.659487,-0.224224,-0.659487,-0.285264,-0.146061,1.203253,0.904705,-2.801180,0.966257,-0.659461
4,1,0,0.442829,1.610635,0.821300,-1.465954,-0.279386,-0.163469,-0.442073,-0.421943,...,-0.657555,-0.196872,-0.657555,-0.285263,-0.146061,0.069855,0.011215,-3.095275,1.049071,-0.658125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.169060,-1.347834,-0.073020,-0.389149,-0.257069,-0.152225,-0.385749,-0.382739,...,-0.537158,-0.209324,-0.537158,-0.282367,-0.146017,-0.295417,-0.322345,0.580916,0.676851,-0.538074
1876,6,1,-0.169060,0.377939,0.038770,-0.478883,-0.249729,-0.150379,-0.263867,-0.376539,...,-0.566952,-0.219464,-0.566952,-0.283311,-0.146033,0.136042,-0.009935,-0.007274,0.676851,-0.567786
1877,6,1,-1.392837,-0.854756,-0.967341,1.315793,-0.249123,-0.148807,-0.203855,-0.348513,...,-0.512414,-0.166815,-0.512414,-0.280506,-0.145960,-0.002798,-0.100433,0.139773,0.676851,-0.513252
1878,6,1,-0.693536,0.377939,-1.079131,1.315793,-0.217467,-0.132307,-0.129628,-0.391167,...,-0.384041,0.621261,-0.384041,-0.273183,-0.145660,-0.065654,-0.006376,-0.007274,0.224671,-0.380832


### Feature Importance

In [322]:
# Separate features and target variable
acc_features = acc_dataset_df.drop(columns=['Class', 'Subject'])
acc_target = acc_dataset_df['Class']
gyro_features = gyro_dataset_df.drop(columns=['Class', 'Subject'])
gyro_target = gyro_dataset_df['Class']
acc_id_groupings = acc_dataset_df['Subject']
gyro_id_groupings = gyro_dataset_df['Subject']

In [323]:
acc_features

,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,CorssCorrelationLag_VTML,CorssCorrelationPeak_VTAP,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
0,0.617654,0.747748,0.597720,0.508189,-0.277193,-0.163081,-0.438910,-0.414596,-0.056982,-0.662803,...,-0.651852,-0.165272,-0.651852,-0.285245,-0.146060,-0.086591,-0.206281,-1.771846,0.959438,-0.652570
1,-1.392837,1.610635,1.380251,0.597923,-0.279359,-0.163483,-0.442177,-0.421079,0.155843,-0.668085,...,-0.659200,-0.224007,-0.659200,-0.285264,-0.146061,0.952947,0.737416,-2.360037,0.966257,-0.659603
2,-1.567663,1.487365,1.268460,0.508189,-0.279558,-0.163548,-0.442086,-0.421402,0.161164,-0.668120,...,-0.659522,-0.224137,-0.659522,-0.285264,-0.146061,1.178809,0.885754,-2.654132,0.966257,-0.659652
3,1.579194,1.610635,1.268460,0.508189,-0.279463,-0.163525,-0.441743,-0.421428,0.161164,-0.668035,...,-0.659487,-0.224224,-0.659487,-0.285264,-0.146061,1.203253,0.904705,-2.801180,0.966257,-0.659461
4,0.442829,1.610635,0.821300,-1.465954,-0.279386,-0.163469,-0.442073,-0.421943,-0.009096,-0.667327,...,-0.657555,-0.196872,-0.657555,-0.285263,-0.146061,0.069855,0.011215,-3.095275,1.049071,-0.658125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,-0.169060,-1.347834,-0.073020,-0.389149,-0.257069,-0.152225,-0.385749,-0.382739,-1.051938,-0.607692,...,-0.537158,-0.209324,-0.537158,-0.282367,-0.146017,-0.295417,-0.322345,0.580916,0.676851,-0.538074
1876,-0.169060,0.377939,0.038770,-0.478883,-0.249729,-0.150379,-0.263867,-0.376539,-1.881956,-0.623584,...,-0.566952,-0.219464,-0.566952,-0.283311,-0.146033,0.136042,-0.009935,-0.007274,0.676851,-0.567786
1877,-1.392837,-0.854756,-0.967341,1.315793,-0.249123,-0.148807,-0.203855,-0.348513,1.842481,-0.580757,...,-0.512414,-0.166815,-0.512414,-0.280506,-0.145960,-0.002798,-0.100433,0.139773,0.676851,-0.513252
1878,-0.693536,0.377939,-1.079131,1.315793,-0.217467,-0.132307,-0.129628,-0.391167,-2.802424,-0.612842,...,-0.384041,0.621261,-0.384041,-0.273183,-0.145660,-0.065654,-0.006376,-0.007274,0.224671,-0.380832


In [324]:
acc_random_forest_model = RandomForestClassifier()
gyro_random_forest_model = RandomForestClassifier()
acc_random_forest_model.fit(acc_features, acc_target)
gyro_random_forest_model.fit(gyro_features, gyro_target)

RandomForestClassifier()

In [325]:
# Get feature importance
acc_feature_importance = acc_random_forest_model.feature_importances_
gyro_feature_importance = gyro_random_forest_model.feature_importances_

In [326]:
acc_feature_importance

array([0.00329797, 0.00309333, 0.00154675, 0.00206462, 0.0045935 ,
       0.00409549, 0.00320885, 0.0045749 , 0.00494639, 0.00506773,
       0.00434596, 0.00227026, 0.00357528, 0.00466122, 0.00253976,
       0.00270142, 0.00166175, 0.00299321, 0.00125526, 0.00228525,
       0.0018625 , 0.00190004, 0.00175553, 0.00232046, 0.0014746 ,
       0.0013969 , 0.00262068, 0.00223954, 0.00314906, 0.00522379,
       0.00303638, 0.00072868, 0.00141509, 0.0010689 , 0.00121657,
       0.00137404, 0.00131466, 0.00218427, 0.0025587 , 0.00237033,
       0.00208105, 0.00250939, 0.00136251, 0.00097829, 0.00156925,
       0.0014881 , 0.00169509, 0.0019328 , 0.00198182, 0.00139244,
       0.00162663, 0.00136111, 0.0017307 , 0.00207147, 0.00178626,
       0.00367837, 0.00725085, 0.00407791, 0.00110755, 0.0013514 ,
       0.00127124, 0.0013973 , 0.00092346, 0.00105455, 0.00206248,
       0.00225942, 0.00195195, 0.00176595, 0.00203423, 0.00194188,
       0.00927961, 0.00467253, 0.00415787, 0.01085976, 0.00181

In [327]:
gyro_feature_importance

array([0.00226461, 0.00234203, 0.00158704, 0.00166825, 0.00653006,
       0.00623849, 0.00794849, 0.0039832 , 0.00270204, 0.0047627 ,
       0.00343166, 0.00285072, 0.00328363, 0.00267823, 0.00297058,
       0.00358142, 0.00202932, 0.00164069, 0.00226869, 0.00249734,
       0.00215432, 0.00194174, 0.00259526, 0.00157821, 0.00162169,
       0.00259758, 0.00253409, 0.00245605, 0.00381429, 0.00495954,
       0.00510013, 0.00134462, 0.00212073, 0.00186014, 0.00154701,
       0.00123082, 0.00095744, 0.00154708, 0.00272436, 0.00220756,
       0.00197982, 0.00199412, 0.00205435, 0.00135876, 0.00156409,
       0.00149091, 0.00193905, 0.00220352, 0.00186171, 0.00156374,
       0.00228346, 0.00227363, 0.0024211 , 0.00268685, 0.00244426,
       0.00420349, 0.00513477, 0.00495603, 0.00174695, 0.00130528,
       0.00200178, 0.00172497, 0.00123228, 0.00213166, 0.0019304 ,
       0.00192903, 0.00298633, 0.00221703, 0.00212129, 0.002646  ,
       0.00417649, 0.00372473, 0.00525681, 0.00381521, 0.00197

In [328]:
acc_best_features = []
gyro_best_features = []

for i in range(len(acc_feature_importance)):
    if acc_feature_importance[i] > 0.0:
        acc_best_features.append(acc_features.columns[i])

for i in range(len(gyro_feature_importance)):
    if gyro_feature_importance[i] > 0.0:
        gyro_best_features.append(gyro_features.columns[i])


In [329]:
filtered_acc_dataset_df = pd.concat([acc_dataset_df.iloc[:, :2], acc_dataset_df[acc_best_features].copy()], axis=1)
filtered_gyro_dataset_df = pd.concat([gyro_dataset_df.iloc[:, :2],gyro_dataset_df[gyro_best_features].copy()], axis=1)

In [330]:
filtered_acc_dataset_df

,Subject,Class,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
0,1,0,0.617654,0.747748,0.597720,0.508189,-0.277193,-0.163081,-0.438910,-0.414596,...,-0.651852,-0.165272,-0.651852,-0.285245,-0.146060,-0.086591,-0.206281,-1.771846,0.959438,-0.652570
1,1,0,-1.392837,1.610635,1.380251,0.597923,-0.279359,-0.163483,-0.442177,-0.421079,...,-0.659200,-0.224007,-0.659200,-0.285264,-0.146061,0.952947,0.737416,-2.360037,0.966257,-0.659603
2,1,0,-1.567663,1.487365,1.268460,0.508189,-0.279558,-0.163548,-0.442086,-0.421402,...,-0.659522,-0.224137,-0.659522,-0.285264,-0.146061,1.178809,0.885754,-2.654132,0.966257,-0.659652
3,1,0,1.579194,1.610635,1.268460,0.508189,-0.279463,-0.163525,-0.441743,-0.421428,...,-0.659487,-0.224224,-0.659487,-0.285264,-0.146061,1.203253,0.904705,-2.801180,0.966257,-0.659461
4,1,0,0.442829,1.610635,0.821300,-1.465954,-0.279386,-0.163469,-0.442073,-0.421943,...,-0.657555,-0.196872,-0.657555,-0.285263,-0.146061,0.069855,0.011215,-3.095275,1.049071,-0.658125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.169060,-1.347834,-0.073020,-0.389149,-0.257069,-0.152225,-0.385749,-0.382739,...,-0.537158,-0.209324,-0.537158,-0.282367,-0.146017,-0.295417,-0.322345,0.580916,0.676851,-0.538074
1876,6,1,-0.169060,0.377939,0.038770,-0.478883,-0.249729,-0.150379,-0.263867,-0.376539,...,-0.566952,-0.219464,-0.566952,-0.283311,-0.146033,0.136042,-0.009935,-0.007274,0.676851,-0.567786
1877,6,1,-1.392837,-0.854756,-0.967341,1.315793,-0.249123,-0.148807,-0.203855,-0.348513,...,-0.512414,-0.166815,-0.512414,-0.280506,-0.145960,-0.002798,-0.100433,0.139773,0.676851,-0.513252
1878,6,1,-0.693536,0.377939,-1.079131,1.315793,-0.217467,-0.132307,-0.129628,-0.391167,...,-0.384041,0.621261,-0.384041,-0.273183,-0.145660,-0.065654,-0.006376,-0.007274,0.224671,-0.380832


In [331]:
filtered_gyro_dataset_df

,Subject,Class,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
0,1,0,-1.434841,-0.089043,-1.377091,-0.918973,-2.081035,-1.393505,-1.720757,-2.025414,...,-2.247603,-0.999462,-2.247603,-0.697426,-0.301041,2.052965,2.272801,-2.533602,2.844971,-2.235480
1,1,0,-1.206018,-0.195019,-0.083083,0.941694,-2.090410,-1.416237,-1.722264,-2.039656,...,-2.245875,-1.000172,-2.245875,-0.697322,-0.301040,2.517236,2.721892,-2.404725,3.116532,-2.272691
2,1,0,-0.862783,-1.254781,-0.471285,1.739122,0.375798,-0.292943,0.668777,-0.129542,...,1.964734,2.097151,1.964734,0.871367,0.250196,0.436805,0.479960,-0.084937,-0.477391,1.543270
3,1,0,-0.748371,-0.724900,1.081524,-0.121544,-0.294936,-0.729608,-0.508218,0.221763,...,1.879856,-0.170724,1.879856,1.224188,0.567989,1.082580,1.077727,-0.213815,-0.477391,1.424152
4,1,0,-0.748371,-0.724900,1.081524,-0.520258,-0.036225,-0.749321,-0.437364,0.650838,...,1.218397,0.172481,1.218397,0.230621,-0.081637,-0.059726,-0.169331,0.172817,-0.314613,0.888175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,6,1,-0.977195,-1.148805,-0.471285,-0.254449,-1.111842,-0.708000,-0.203888,-1.286660,...,-0.118216,0.194818,-0.118216,-0.012726,-0.120209,0.982122,0.997675,-1.244831,0.883994,-0.289412
2055,6,1,-0.633960,-1.148805,-1.506492,-0.254449,-1.511486,-0.864506,-0.758434,-1.171306,...,-0.712570,-1.001458,-0.712570,-0.363895,-0.241271,0.855779,0.838980,-0.987077,0.883994,-0.735938
2056,6,1,-0.748371,-1.148805,1.469727,-0.121544,-1.171761,-0.574156,-0.166071,-1.024269,...,0.338616,0.425363,0.338616,-0.004442,-0.175226,-0.445744,-0.629712,-1.115954,1.115099,0.036538
2057,6,1,-0.405136,-1.254781,-1.506492,-0.254449,-0.912437,0.335347,0.745596,-1.053510,...,0.905783,0.369655,0.905783,0.520854,0.044095,0.099458,-0.100948,-0.858200,1.115099,0.687802


### Correlation Selection

In [332]:
acc_corr = filtered_acc_dataset_df.drop(columns=['Subject']).corr()
gyro_corr = filtered_gyro_dataset_df.drop(columns=['Subject']).corr()

In [333]:
acc_corr

,Class,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,CorssCorrelationLag_VTML,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
Class,1.000000,0.071103,-0.014942,0.035174,0.029080,0.074567,0.059947,0.091773,0.010877,0.009457,...,0.045063,-0.030661,0.045063,-0.020587,-0.043304,-0.082436,-0.053874,0.266133,-0.083044,0.045174
DominantFreq_VT,0.071103,1.000000,0.287872,0.411640,0.346239,-0.125137,-0.104903,-0.169717,-0.146611,0.034360,...,-0.141517,-0.072151,-0.141517,-0.121576,-0.095920,0.234358,0.253466,0.157761,0.074291,-0.141566
DominantFreq_ML,-0.014942,0.287872,1.000000,0.349580,0.156553,-0.137228,-0.079420,-0.186502,-0.112258,0.034897,...,-0.070854,-0.037553,-0.070854,-0.049601,-0.046595,0.113566,0.105703,-0.094247,0.089535,-0.070472
DominantFreq_AP,0.035174,0.411640,0.349580,1.000000,0.299412,-0.121205,-0.098365,-0.204293,-0.139303,-0.040785,...,-0.192907,-0.081502,-0.192907,-0.140342,-0.091394,0.231490,0.265429,0.099945,0.164294,-0.192788
DominantFreq_Magnitude,0.029080,0.346239,0.156553,0.299412,1.000000,-0.070660,-0.061642,-0.073169,-0.064642,-0.050878,...,-0.178983,-0.080402,-0.178983,-0.122432,-0.089314,0.271781,0.292703,0.348251,0.179151,-0.178823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SkewnessPSD_Plom_Magnitude,-0.082436,0.234358,0.113566,0.231490,0.271781,0.081940,-0.018392,0.018540,-0.034365,-0.094527,...,-0.238930,-0.106512,-0.238930,-0.057479,0.014536,1.000000,0.983090,0.111286,0.417422,-0.238384
KurtosisPSD_Plom_Magnitude,-0.053874,0.253466,0.105703,0.265429,0.292703,0.078655,-0.012576,0.020874,-0.038160,-0.113495,...,-0.233746,-0.086410,-0.233746,-0.072460,-0.002437,0.983090,1.000000,0.170087,0.385905,-0.233279
ZeroCrossingRate_Magnitude,0.266133,0.157761,-0.094247,0.099945,0.348251,-0.086506,-0.103723,-0.069341,-0.065809,-0.016314,...,-0.118895,-0.034908,-0.118895,-0.116636,-0.092186,0.111286,0.170087,1.000000,0.066861,-0.119376
ModeSignal_Magnitude,-0.083044,0.074291,0.089535,0.164294,0.179151,-0.023072,-0.038757,-0.160103,-0.174752,-0.007853,...,-0.816657,-0.211662,-0.816657,-0.530642,-0.318274,0.417422,0.385905,0.066861,1.000000,-0.815876


In [334]:
gyro_corr

,Class,DominantFreq_VT,DominantFreq_ML,DominantFreq_AP,DominantFreq_Magnitude,CorssCorrelationSum_VTML,CorssCorrelationSum_VTAP,CorssCorrelationSum_MLAP,CorssCorrelationPeak_VTML,CorssCorrelationLag_VTML,...,MeanPSD_Plomb_Magnitude,MedianPSD_Plom_Magnitude,SumPSD_Plom_Magnitude,Moment2ndPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude,ModeSignal_Magnitude,EnergySignal_Magnitude
Class,1.000000,0.006352,-0.031728,-0.018058,0.076741,-0.066162,-0.153018,-0.097257,-0.072464,-0.008205,...,-0.170965,-0.081289,-0.170965,-0.021036,0.029526,0.009703,0.017484,0.005220,0.106498,-0.101430
DominantFreq_VT,0.006352,1.000000,0.242069,0.338420,0.126235,-0.048765,-0.220322,-0.160712,0.080432,0.037079,...,-0.088473,-0.067361,-0.088473,-0.095632,-0.056245,-0.140797,-0.127394,0.208012,0.021850,-0.092210
DominantFreq_ML,-0.031728,0.242069,1.000000,0.273331,0.099235,-0.037871,-0.146511,-0.107816,0.066225,0.027798,...,-0.100851,-0.047246,-0.100851,-0.110557,-0.078294,-0.126292,-0.112024,0.145924,-0.020870,-0.110840
DominantFreq_AP,-0.018058,0.338420,0.273331,1.000000,0.100081,-0.064418,-0.198025,-0.153901,0.066800,-0.001257,...,-0.022709,-0.047245,-0.022709,-0.049316,-0.033985,-0.089161,-0.080669,0.142443,-0.031109,-0.027910
DominantFreq_Magnitude,0.076741,0.126235,0.099235,0.100081,1.000000,0.039186,-0.047434,-0.006472,0.062827,-0.030609,...,-0.197814,-0.167716,-0.197814,-0.128252,-0.053615,-0.227935,-0.196306,0.217070,0.031440,-0.178453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SkewnessPSD_Plom_Magnitude,0.009703,-0.140797,-0.126292,-0.089161,-0.227935,-0.193241,0.005241,-0.078268,-0.227353,-0.018423,...,0.406073,0.112104,0.406073,0.561028,0.413727,1.000000,0.991191,-0.543720,0.027324,0.421764
KurtosisPSD_Plom_Magnitude,0.017484,-0.127394,-0.112024,-0.080669,-0.196306,-0.189237,0.004061,-0.074489,-0.218479,-0.020843,...,0.368227,0.106225,0.368227,0.549825,0.418505,0.991191,1.000000,-0.507980,0.032391,0.389984
ZeroCrossingRate_Magnitude,0.005220,0.208012,0.145924,0.142443,0.217070,0.319356,0.034853,0.131063,0.419583,0.021343,...,-0.232550,-0.100587,-0.232550,-0.370634,-0.258964,-0.543720,-0.507980,1.000000,-0.207795,-0.421815
ModeSignal_Magnitude,0.106498,0.021850,-0.020870,-0.031109,0.031440,-0.396787,-0.302145,-0.330013,-0.430123,0.010932,...,-0.476630,-0.181596,-0.476630,-0.182030,-0.086185,0.027324,0.032391,-0.207795,1.000000,-0.391182


In [335]:
acc_feature_corr_matrix = acc_corr.drop(columns=['Class'])
acc_target_corr = acc_feature_corr_matrix.iloc[[0]].copy()
acc_feature_corr_matrix = acc_feature_corr_matrix.drop(acc_feature_corr_matrix.index[0])
gyro_feature_corr_matrix = gyro_corr.drop(columns=['Class'])
gyro_target_corr = gyro_feature_corr_matrix.iloc[[0]].copy()
gyro_feature_corr_matrix = gyro_feature_corr_matrix.drop(gyro_feature_corr_matrix.index[0])

In [336]:
acc_corr_redundant_features = get_corr_redundant_features(acc_feature_corr_matrix, acc_target_corr)
gyro_corr_redundant_features = get_corr_redundant_features(gyro_feature_corr_matrix, gyro_target_corr)

In [337]:
print(len(acc_corr_redundant_features))
acc_corr_redundant_features

173


['CorssCorrelationSum_VTML',
 'CorssCorrelationPeak_MLAP',
 'DominantFreq_ML',
 'DominantFreq_CorrVTAP',
 'DominantFreq_CorrVTML',
 'Amplitude_ML',
 'Amplitude_AP',
 'AmplitudeNorm_AP',
 'Width_VT',
 'Amplitude_VT',
 'Slope_VT',
 'AmplitudeNorm_ML',
 'Range_VT',
 'Range_AP',
 'MomentPSD_2nd_VT',
 'MomentPSD_2nd_AP',
 'MomentPSD_2nd_ML',
 'MomentPSD_3rd_AP',
 'SkewnessPSD_VT',
 'KurtosisPSD_VT',
 'KurtosisPSD_ML',
 'KurtosisPSD_AP',
 'MeanPower_VT',
 'MeanPower_ML',
 'MeanPower_AP',
 'MedianPower_VT',
 'WidthNorm_VT',
 'WidthNorm_ML',
 'Width_AP',
 'SlopeNorm_VT',
 'SlopeNorm_ML',
 'SlopeNorm_AP',
 'RangeNorm_VT',
 'Range_ML',
 'RangeNorm_AP',
 'IntegratedPower_VT',
 'IntegratedPower_ML',
 'IntegratedPower_AP',
 'SumPSD_VT',
 'SumPSD_ML',
 'SumPSD_AP',
 'IndexHarmonicity_VT',
 'HarmonicRatio_ML',
 'HarmonicRatio_AP',
 'Median_SigComplete_VT',
 'Range_SigComplete_VT',
 'RMS_SigComplete_VT',
 'RangeNorm_ML',
 'FirstDerivativeMax_SigComplete_VT',
 'FirstDerivative_Skweness_SigComplete_VT',

In [338]:
print(len(gyro_corr_redundant_features))
gyro_corr_redundant_features

173


['CorssCorrelationSum_VTML',
 'CorssCorrelationSum_MLAP',
 'CorssCorrelationSum_VTAP',
 'CorssCorrelationPeak_VTML',
 'CorssCorrelationPeak_MLAP',
 'DominantFreq_VT',
 'DominantFreq_CorrVTAP',
 'DominantFreq_AP',
 'DominantFreq_CorrMLAP',
 'Amplitude_VT',
 'Amplitude_ML',
 'Amplitude_AP',
 'AmplitudeNorm_VT',
 'Slope_ML',
 'Slope_AP',
 'Range_ML',
 'Range_AP',
 'MomentPSD_2nd_VT',
 'MomentPSD_3rd_ML',
 'MomentPSD_2nd_AP',
 'SkewnessPSD_ML',
 'SkewnessPSD_VT',
 'KurtosisPSD_ML',
 'SkewnessPSD_AP',
 'MeanPower_ML',
 'MeanPower_AP',
 'Width_VT',
 'WidthNorm_ML',
 'WidthNorm_AP',
 'Slope_VT',
 'SlopeNorm_VT',
 'SlopeNorm_ML',
 'SlopeNorm_AP',
 'Range_VT',
 'RangeNorm_ML',
 'RangeNorm_AP',
 'MeanPower_VT',
 'IntegratedPower_ML',
 'IntegratedPower_AP',
 'SumPSD_VT',
 'SumPSD_ML',
 'SumPSD_AP',
 'HarmonicRatio_ML',
 'HarmonicRatio_AP',
 'RangeNorm_VT',
 'RMS_SigComplete_VT',
 'RMSratio_SigComplete_VT',
 'Range_SigComplete_VT',
 'FirstDerivative_Moment2nd_SigComplete_VT',
 'FirstDerivative_Skw

In [339]:
filtered_acc_dataset_df = filtered_acc_dataset_df.drop(columns=acc_corr_redundant_features)
filtered_gyro_dataset_df = filtered_gyro_dataset_df.drop(columns=gyro_corr_redundant_features)

In [340]:
filtered_acc_dataset_df

,Subject,Class,CorssCorrelationSum_VTAP,CorssCorrelationLag_VTML,CorssCorrelationPeak_VTAP,CorssCorrelationLag_VTAP,CorssCorrelationLag_MLAP,AmplitudeNorm_VT,Width_ML,Slope_ML,...,MaxFreqSpectrum_Below5Hz_Magnitude,NumPeaksSpectrum_Below5Hz_Magnitude,RelativePower_5To10Hz_Magnitude,RelativePower_BelowDomFr_Magnitude,RelativePower_AboveDomFr_Magnitude,CentroidSpMax_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude
0,1,0,-0.163081,-0.056982,-0.662803,0.200520,-0.138776,0.023358,0.637970,-1.078878,...,-0.116140,1.233052,1.224695,-0.156583,2.557185,-1.002728,-0.165272,-0.146060,-0.086591,-1.771846
1,1,0,-0.163483,0.155843,-0.668085,0.234964,0.046541,-0.840076,0.041131,0.699628,...,0.408305,-1.087407,1.251751,-0.063859,-0.153913,-0.077458,-0.224007,-0.146061,0.952947,-2.360037
2,1,0,-0.163548,0.161164,-0.668120,0.234964,0.046541,-0.789907,-0.537371,3.319331,...,-2.039106,-1.087407,1.249730,0.167928,-0.168584,0.145096,-0.224137,-0.146061,1.178809,-2.654132
3,1,0,-0.163525,0.161164,-0.668035,0.234964,0.046541,0.958601,-0.505722,0.145211,...,-2.039106,-1.087407,1.052092,0.285549,-0.144953,0.685840,-0.224224,-0.146061,1.203253,-2.801180
4,1,0,-0.163469,-0.009096,-0.667327,0.092265,-0.138776,1.120842,-0.704000,-0.773438,...,-1.951699,1.233052,-1.596697,-0.016023,-0.200118,3.192919,-0.196872,-0.146061,0.069855,-3.095275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.152225,-1.051938,-0.607692,-0.498214,-0.078548,-1.565972,-0.444224,-1.106786,...,-0.465771,1.233052,0.163099,-0.191222,-0.200118,-0.263290,-0.209324,-0.146017,-0.295417,0.580916
1876,6,1,-0.150379,-1.881956,-0.623584,-1.718538,0.009478,-0.397318,0.740223,-0.371780,...,-0.553178,0.072823,0.996511,-0.178239,-0.200118,-0.541632,-0.219464,-0.146033,0.136042,-0.007274
1877,6,1,-0.148807,1.842481,-0.580757,1.725925,0.018744,-0.026732,-0.155491,-0.201656,...,1.194972,0.072823,0.894499,-0.184609,0.259420,1.207324,-0.166815,-0.145960,-0.002798,0.139773
1878,6,1,-0.132307,-2.802424,-0.612842,-0.266943,-0.078548,0.081518,-1.067177,-0.246587,...,1.194972,0.072823,0.132428,-0.083886,2.964788,2.514737,0.621261,-0.145660,-0.065654,-0.007274


In [341]:
filtered_gyro_dataset_df

,Subject,Class,DominantFreq_Magnitude,CorssCorrelationLag_VTML,CorssCorrelationLag_VTAP,CorssCorrelationLag_MLAP,AmplitudeNorm_ML,AmplitudeNorm_AP,Width_ML,Width_AP,...,Std_SigComplete_Magnitude,Skewness_SigComplete_Magnitude,Percentile25_SigComplete_Magnitude,MaxFreqSpectrum_Below5Hz_Magnitude,NumPeaksSpectrum_Below5Hz_Magnitude,RelativePower_Below5Hz_Magnitude,CentroidSpMax_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude
0,1,0,-0.918973,-0.185585,3.572217,-0.450055,0.931338,0.275375,0.232719,0.687193,...,-3.550868,-0.985179,1.048679,-0.637165,-0.386252,-2.247382,-1.283696,-0.999462,-0.301041,2.272801
1,1,0,0.941694,-0.138234,-3.960020,-0.436952,2.945881,0.361647,-0.425020,-0.504999,...,-3.541093,-0.919764,1.159250,-0.637165,-1.895346,-2.401791,-1.814965,-1.000172,-0.301040,2.721892
2,1,0,1.739122,0.037639,0.259930,1.404084,2.260086,-0.184135,-0.566651,-1.098881,...,1.611685,-0.078815,-1.605170,-0.465109,1.122843,-0.511308,0.764111,2.097151,0.250196,0.479960
3,1,0,-0.121544,0.051168,0.266703,0.185463,2.455305,-0.588941,-0.108484,-0.544062,...,1.496111,0.082070,-1.621792,-0.465109,1.122843,-0.713828,-0.258488,-0.170724,0.567989,1.077727
4,1,0,-0.520258,0.044404,0.266703,0.185463,1.117024,-0.436505,-0.598245,-0.777859,...,1.127797,0.092284,-0.883628,-0.293054,-0.386252,0.242117,-1.066616,0.172481,-0.081637,-0.169331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,6,1,-0.254449,-0.368223,-0.004240,-3.175574,-2.230008,-1.416072,1.255105,0.818296,...,0.027621,1.734425,-0.346817,-0.293054,1.122843,0.432625,-0.749803,0.194818,-0.120209,0.997675
2055,6,1,-0.254449,-0.361459,0.016080,0.454083,-1.236432,-1.584291,-1.163135,-0.603245,...,-0.604758,2.085799,-0.346817,-0.465109,-0.386252,0.203724,-0.553956,-1.001458,-0.241271,0.838980
2056,6,1,-0.121544,-0.354694,0.002533,0.447532,1.054683,-0.254752,1.061411,0.715187,...,0.394727,1.875773,-0.287707,0.395167,1.122843,1.628756,0.711731,0.425363,-0.175226,-0.629712
2057,6,1,-0.254449,1.579916,0.009307,-1.478676,1.230151,0.519923,1.412938,0.302753,...,0.841233,0.788765,0.206460,-0.120999,-0.386252,1.868931,0.526437,0.369655,0.044095,-0.100948


### Backwards Elimination

In [358]:
acc_features = filtered_acc_dataset_df.iloc[:, 2:]
gyro_features = filtered_gyro_dataset_df.iloc[:, 2:]

In [359]:
acc_features

,CorssCorrelationSum_VTAP,CorssCorrelationLag_VTML,CorssCorrelationPeak_VTAP,CorssCorrelationLag_VTAP,CorssCorrelationLag_MLAP,AmplitudeNorm_VT,Width_ML,Slope_ML,Slope_AP,MomentPSD_3rd_VT,...,MaxFreqSpectrum_Below5Hz_Magnitude,NumPeaksSpectrum_Below5Hz_Magnitude,RelativePower_5To10Hz_Magnitude,RelativePower_BelowDomFr_Magnitude,RelativePower_AboveDomFr_Magnitude,CentroidSpMax_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude
0,-0.163081,-0.056982,-0.662803,0.200520,-0.138776,0.023358,0.637970,-1.078878,-1.064460,-0.432420,...,-0.116140,1.233052,1.224695,-0.156583,2.557185,-1.002728,-0.165272,-0.146060,-0.086591,-1.771846
1,-0.163483,0.155843,-0.668085,0.234964,0.046541,-0.840076,0.041131,0.699628,1.520720,0.071743,...,0.408305,-1.087407,1.251751,-0.063859,-0.153913,-0.077458,-0.224007,-0.146061,0.952947,-2.360037
2,-0.163548,0.161164,-0.668120,0.234964,0.046541,-0.789907,-0.537371,3.319331,3.805847,-0.008747,...,-2.039106,-1.087407,1.249730,0.167928,-0.168584,0.145096,-0.224137,-0.146061,1.178809,-2.654132
3,-0.163525,0.161164,-0.668035,0.234964,0.046541,0.958601,-0.505722,0.145211,0.301036,-0.420876,...,-2.039106,-1.087407,1.052092,0.285549,-0.144953,0.685840,-0.224224,-0.146061,1.203253,-2.801180
4,-0.163469,-0.009096,-0.667327,0.092265,-0.138776,1.120842,-0.704000,-0.773438,-0.886657,-0.433069,...,-1.951699,1.233052,-1.596697,-0.016023,-0.200118,3.192919,-0.196872,-0.146061,0.069855,-3.095275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,-0.152225,-1.051938,-0.607692,-0.498214,-0.078548,-1.565972,-0.444224,-1.106786,-1.041698,-0.422312,...,-0.465771,1.233052,0.163099,-0.191222,-0.200118,-0.263290,-0.209324,-0.146017,-0.295417,0.580916
1876,-0.150379,-1.881956,-0.623584,-1.718538,0.009478,-0.397318,0.740223,-0.371780,0.203519,0.358818,...,-0.553178,0.072823,0.996511,-0.178239,-0.200118,-0.541632,-0.219464,-0.146033,0.136042,-0.007274
1877,-0.148807,1.842481,-0.580757,1.725925,0.018744,-0.026732,-0.155491,-0.201656,0.719357,-0.100353,...,1.194972,0.072823,0.894499,-0.184609,0.259420,1.207324,-0.166815,-0.145960,-0.002798,0.139773
1878,-0.132307,-2.802424,-0.612842,-0.266943,-0.078548,0.081518,-1.067177,-0.246587,-0.593229,-0.425358,...,1.194972,0.072823,0.132428,-0.083886,2.964788,2.514737,0.621261,-0.145660,-0.065654,-0.007274


In [360]:
gyro_features

,DominantFreq_Magnitude,CorssCorrelationLag_VTML,CorssCorrelationLag_VTAP,CorssCorrelationLag_MLAP,AmplitudeNorm_ML,AmplitudeNorm_AP,Width_ML,Width_AP,MomentPSD_2nd_ML,MomentPSD_3rd_VT,...,Std_SigComplete_Magnitude,Skewness_SigComplete_Magnitude,Percentile25_SigComplete_Magnitude,MaxFreqSpectrum_Below5Hz_Magnitude,NumPeaksSpectrum_Below5Hz_Magnitude,RelativePower_Below5Hz_Magnitude,CentroidSpMax_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,KurtosisPSD_Plom_Magnitude
0,-0.918973,-0.185585,3.572217,-0.450055,0.931338,0.275375,0.232719,0.687193,1.488946,-0.522955,...,-3.550868,-0.985179,1.048679,-0.637165,-0.386252,-2.247382,-1.283696,-0.999462,-0.301041,2.272801
1,0.941694,-0.138234,-3.960020,-0.436952,2.945881,0.361647,-0.425020,-0.504999,4.531830,-0.378323,...,-3.541093,-0.919764,1.159250,-0.637165,-1.895346,-2.401791,-1.814965,-1.000172,-0.301040,2.721892
2,1.739122,0.037639,0.259930,1.404084,2.260086,-0.184135,-0.566651,-1.098881,0.023189,-0.502380,...,1.611685,-0.078815,-1.605170,-0.465109,1.122843,-0.511308,0.764111,2.097151,0.250196,0.479960
3,-0.121544,0.051168,0.266703,0.185463,2.455305,-0.588941,-0.108484,-0.544062,3.517353,-0.415202,...,1.496111,0.082070,-1.621792,-0.465109,1.122843,-0.713828,-0.258488,-0.170724,0.567989,1.077727
4,-0.520258,0.044404,0.266703,0.185463,1.117024,-0.436505,-0.598245,-0.777859,0.002377,-0.426310,...,1.127797,0.092284,-0.883628,-0.293054,-0.386252,0.242117,-1.066616,0.172481,-0.081637,-0.169331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,-0.254449,-0.368223,-0.004240,-3.175574,-2.230008,-1.416072,1.255105,0.818296,-0.914137,-0.546034,...,0.027621,1.734425,-0.346817,-0.293054,1.122843,0.432625,-0.749803,0.194818,-0.120209,0.997675
2055,-0.254449,-0.361459,0.016080,0.454083,-1.236432,-1.584291,-1.163135,-0.603245,-1.042166,-0.468341,...,-0.604758,2.085799,-0.346817,-0.465109,-0.386252,0.203724,-0.553956,-1.001458,-0.241271,0.838980
2056,-0.121544,-0.354694,0.002533,0.447532,1.054683,-0.254752,1.061411,0.715187,-0.475118,0.718958,...,0.394727,1.875773,-0.287707,0.395167,1.122843,1.628756,0.711731,0.425363,-0.175226,-0.629712
2057,-0.254449,1.579916,0.009307,-1.478676,1.230151,0.519923,1.412938,0.302753,3.049587,-0.195433,...,0.841233,0.788765,0.206460,-0.120999,-0.386252,1.868931,0.526437,0.369655,0.044095,-0.100948


In [361]:
# Initialize the logistic regression model
model = LogisticRegression()

# Initialize GroupKFold with the number of splits
gkf = GroupKFold(n_splits=6)

# Initialize RFECV with logistic regression model and GroupKFold
acc_rfecv = RFECV(estimator=model, cv=gkf)
gyro_rfecv = RFECV(estimator=model, cv=gkf)

# Fit RFECV to the data
acc_rfecv.fit(acc_features, acc_target, groups=acc_id_groupings)
gyro_rfecv.fit(gyro_features, gyro_target, groups=gyro_id_groupings)

# Get the selected features
acc_selected_features = acc_features.columns[acc_rfecv.support_]
gyro_selected_features = gyro_features.columns[gyro_rfecv.support_]

In [362]:
acc_selected_features

Index(['CorssCorrelationSum_VTAP', 'CorssCorrelationPeak_VTAP',
       'IndexHarmonicity_ML', 'FirstDerivativeMean_SigComplete_VT',
       'FirstDerivative_Moment3rd_SigComplete_VT',
       'FirstDerivativeLogRatio_SigComplete_ML',
       'FirstDerivativeMean_SigComplete_AP',
       'FirstDerivative_Skweness_SigComplete_AP',
       'FirstDerivativeRatio_SigComplete_AP',
       'FirstDerivativeMax_SigComplete_Magnitude', 'RelativePower_Below5Hz_VT',
       'RelativePower_BelowDomFr_VT', 'Skewness_SigComplete_ML',
       'Kurtosis_SigComplete_ML', 'Percentile75_SigComplete_ML',
       'MaxFreqSpectrum_Below5Hz_ML', 'RelativePower_AboveDomFr_ML',
       'Moment3rdPSD_Plom_ML', 'Max_SigComplete_AP',
       'InterQuartileRange_SigComplete_AP', 'Percentile75_SigComplete_AP',
       'ZeroCrossingRate_AP', 'Mean_SigComplete_Magnitude',
       'Skewness_SigComplete_Magnitude', 'Percentile25_SigComplete_Magnitude',
       'MaxFreqSpectrum_Below5Hz_Magnitude', 'RelativePower_5To10Hz_Magnitude',
 

In [363]:
gyro_selected_features

Index(['RelativePower_Below5Hz_Magnitude'], dtype='object')

In [364]:
filtered_acc_dataset_df = pd.concat([acc_dataset_df.iloc[:, :2], acc_dataset_df[acc_selected_features].copy()], axis=1)
filtered_gyro_dataset_df = pd.concat([gyro_dataset_df.iloc[:, :2], gyro_dataset_df[gyro_selected_features].copy()], axis=1)

In [365]:
filtered_acc_dataset_df

,Subject,Class,CorssCorrelationSum_VTAP,CorssCorrelationPeak_VTAP,IndexHarmonicity_ML,FirstDerivativeMean_SigComplete_VT,FirstDerivative_Moment3rd_SigComplete_VT,FirstDerivativeLogRatio_SigComplete_ML,FirstDerivativeMean_SigComplete_AP,FirstDerivative_Skweness_SigComplete_AP,...,Mean_SigComplete_Magnitude,Skewness_SigComplete_Magnitude,Percentile25_SigComplete_Magnitude,MaxFreqSpectrum_Below5Hz_Magnitude,RelativePower_5To10Hz_Magnitude,RelativePower_BelowDomFr_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude
0,1,0,-0.163081,-0.662803,0.528986,0.475757,-0.062359,-0.136782,-0.409614,-1.334350,...,-0.035799,1.275514,1.444325,-0.116140,1.224695,-0.156583,-0.165272,-0.146060,-0.086591,-1.771846
1,1,0,-0.163483,-0.668085,-0.364654,0.090667,-0.110018,-0.233595,-0.054282,-1.066276,...,-0.281341,-0.695417,1.432455,0.408305,1.251751,-0.063859,-0.224007,-0.146061,0.952947,-2.360037
2,1,0,-0.163548,-0.668120,-0.362345,-0.033453,-0.110054,-0.112613,-0.050146,-1.160794,...,-0.404884,-0.490985,1.412446,-2.039106,1.249730,0.167928,-0.224137,-0.146061,1.178809,-2.654132
3,1,0,-0.163525,-0.668035,-0.376434,0.009309,-0.110064,-0.227149,-0.042921,-1.174912,...,-0.463610,-0.346706,1.393891,-2.039106,1.052092,0.285549,-0.224224,-0.146061,1.203253,-2.801180
4,1,0,-0.163469,-0.667327,13.995720,-0.002560,-0.109835,-0.698540,0.162228,-0.055198,...,-0.208414,2.387815,1.407141,-1.951699,-1.596697,-0.016023,-0.196872,-0.146061,0.069855,-3.095275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.152225,-0.607692,-0.416798,-0.867979,-0.047874,0.015603,-0.046169,0.002909,...,0.185887,0.225318,0.462994,-0.465771,0.163099,-0.191222,-0.209324,-0.146017,-0.295417,0.580916
1876,6,1,-0.150379,-0.623584,-0.154285,-0.524313,-0.062139,-0.028567,0.040108,0.172652,...,0.169964,0.472963,0.627831,-0.553178,0.996511,-0.178239,-0.219464,-0.146033,0.136042,-0.007274
1877,6,1,-0.148807,-0.580757,-0.314912,0.828930,-0.047780,-0.325827,-0.340791,0.078664,...,0.280245,0.255260,0.389285,1.194972,0.894499,-0.184609,-0.166815,-0.145960,-0.002798,0.139773
1878,6,1,-0.132307,-0.612842,-0.367696,1.526718,-0.039679,-0.599088,-2.453837,-3.100024,...,1.338794,0.849419,0.387508,1.194972,0.132428,-0.083886,0.621261,-0.145660,-0.065654,-0.007274


In [366]:
filtered_gyro_dataset_df

,Subject,Class,RelativePower_Below5Hz_Magnitude
0,1,0,-2.247382
1,1,0,-2.401791
2,1,0,-0.511308
3,1,0,-0.713828
4,1,0,0.242117
...,...,...,...
2054,6,1,0.432625
2055,6,1,0.203724
2056,6,1,1.628756
2057,6,1,1.868931


### Saving to CSV

In [367]:
filtered_acc_dataset_df.to_csv(final_dataset_path + 'PatchTable_Acceleration_Filtered.csv', index=False)
filtered_gyro_dataset_df.to_csv(final_dataset_path + 'PatchTable_Gyroscope_Filtered.csv', index=False)